# OOI Data to Zarr in Azure Blob Example
This is an example Python notebook showing how to move OOI data into Azure Blob using Xarray and the ABSStore data store in Zarr. Here we move one month of a velocity meter into a test blob storage account on Azure, set the container to public read-only access, and then annonymously load those data into Xarray from Azure. We show a plot of the velocity data.

## Request data from OOI using M2M
Here we load our OOINet credentials and submit a request for data using the machine-to-machine (M2M) interface

In [ ]:
# Load ooinet credentials
import yaml
with open('.ooinet_credentials.yaml', 'r') as f:
    ooinet_credentials = yaml.load(f)

In [ ]:
# build M2M request using instrument identifiers
subsite = 'RS01SUM1'
node = 'LJ01B'
sensor = '12-VEL3DB104'
method = 'streamed'
stream = 'vel3d_b_sample' # 3-D velocity meter
beginDT = '2015-01-01T00:00:00.000Z'
endDT = '2015-01-02T00:00:00.000Z'

base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'
data_request_url ='/'.join((base_url,subsite,node,sensor,method,stream))
params = {'beginDT':beginDT, 'endDT':endDT}
data_request_url

In [ ]:
# submit request
import requests
r = requests.get(data_request_url, params=params, auth=(ooinet_credentials['username'], ooinet_credentials['token']))
data = r.json()
data['allURLs'][0]

## Load data into an Xarray dataset
Xarray can load the generated NetCDF file directly and lazily from the OpenDAP server. 

In [ ]:
netcdf_url = 'https://opendap.oceanobservatories.org/thredds/dodsC/ooi/crone@ldeo.columbia.edu/20190101T160240-RS01SUM1-LJ01B-12-VEL3DB104-streamed-vel3d_b_sample/deployment0001_RS01SUM1-LJ01B-12-VEL3DB104-streamed-vel3d_b_sample_20150101T000000-20150131T235814.nc'

In [ ]:
import xarray as xr
ds = xr.open_dataset(netcdf_url, drop_variables = ['id',
                                                   'deployment',
                                                   'date_time_string',
                                                   'driver_timestamp',
                                                   'ingestion_timestamp',
                                                   'internal_timestamp',
                                                   'mag_comp_x',
                                                   'mag_comp_y',
                                                   'pitch',
                                                   'port_timestamp',
                                                   'preferred_timestamp',
                                                   'provenance',
                                                   'roll',
                                                   'temperature',
                                                   'velocity_beam_a',
                                                   'velocity_beam_b',
                                                   'velocity_beam_c',
                                                   'velocity_beam_d',
                                                   'eastward_turbulent_velocity',
                                                   'northward_turbulent_velocity',
                                                   'upward_turbulent_velocity',
                                                   'temperature_qc_executed',
                                                   'temperature_qc_results',
                                                   'turbulent_velocity_east_qc_executed',
                                                   'turbulent_velocity_east_qc_results',
                                                   'turbulent_velocity_north_qc_executed',
                                                   'turbulent_velocity_north_qc_results',
                                                   'turbulent_velocity_up_qc_executed',
                                                   'turbulent_velocity_up_qc_results',
                                                   'eastward_turbulent_velocity_qc_executed',
                                                   'eastward_turbulent_velocity_qc_results',
                                                   'northward_turbulent_velocity_qc_executed',
                                                   'northward_turbulent_velocity_qc_results',
                                                   'upward_turbulent_velocity_qc_executed',
                                                   'upward_turbulent_velocity_qc_results'])
ds = ds.swap_dims({'obs': 'time'})
ds = ds.reset_coords(names = ['obs', 'lat', 'lon'], drop=True)
ds

## Save dataset to Azure blob using Zarr

In [ ]:
# load Azure storage account credentials
with open('.azure_credentials_ooitest.yaml', 'r') as f:
    credentials = yaml.load(f)
azure_storage_account_name = credentials['azure_storage_account_name']
azure_storage_account_key = credentials['azure_storage_account_key']

In [ ]:
# create a container for the velocity data
from azure.storage.blob import BlockBlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
container_name = 'velocity'
blob_service.create_container(container_name);

In [ ]:
# delete all blobs in container
blobs = blob_service.list_blobs(container_name)
for a in blobs:
    blob_service.delete_blob(container_name = container_name, blob_name = a.name)

In [ ]:
# create an absstore object for the to_zarr method
import zarr
filename = 'velocity.zarr'
absstore = zarr.storage.ABSStore(container_name, filename, azure_storage_account_name, azure_storage_account_key)

In [ ]:
# save the data to Azure
ds.to_zarr(absstore);

## Read data from Azure Blob using anonymous read-only access

In [ ]:
import zarr
container_name = 'velocity'
filename = 'velocity.zarr'
absstore_anon = zarr.storage.ABSStore(container_name, filename,  'ooitest')

In [ ]:
import xarray as xr
ds = xr.open_zarr(absstore_anon)
#df = ds.to_dataframe();
ds

In [ ]:
df.head()

In [ ]:
zarr.__version__

In [ ]:
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [ ]:
#plot a subset of the data
df.hvplot(datashade=True)

In [ ]:
!conda list | grep azure